In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
import mplfinance as mpf
from sklearn.linear_model import LinearRegression
import itertools
import random

In [2]:
def download_data(stock):
    df = yf.download(stock, start="2024-03-01", progress=False)
    df.dropna(inplace=True)
    return df

def modifications(df):
    # Step 1: flatten the MultiIndex
    df.columns = [col[0] for col in df.columns]

    ohlc_cols = ["Open", "High", "Low", "Close", "Volume"]
    for col in ohlc_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Ensure your Date column is datetime and set as index
    df.index = pd.to_datetime(df.index)
    return df

def rolling_slope(series, window):
    """
    Returns rolling linear regression slope
    """
    slopes = np.full(len(series), np.nan)

    x = np.arange(window)

    for i in range(window, len(series)):
        y = series.iloc[i-window:i].values
        slope = np.polyfit(x, y, 1)[0]
        slopes[i] = slope

    return slopes

def atr_mult_from_percentile(p):
    """
    p : volatility percentile (0–1)
    """
    p = max(0, min(p, 1))
    return 2.8 + 2.2 * (p ** 2.0)

In [3]:
def build_indicators(df, slope50_w, slope200_w1, slope200_w2):
    df = df.copy()

    # EMA & DMA
    df["EMA50"]  = df["Close"].ewm(span=50).mean()
    df["DMA200"] = df["Close"].rolling(200).mean()
    df["DMA50"] = df["Close"].rolling(50).mean()

    # Rolling linear regression slopes
    df["EMA50_SLOPE"] = rolling_slope(df["EMA50"], slope50_w)
    df["DMA200_SLOPE1"] = rolling_slope(df["DMA200"], slope200_w1)
    df["DMA200_SLOPE2"] = rolling_slope(df["DMA200"], slope200_w2)
    df["DMA50_SLOPE"] = rolling_slope(df["DMA50"], slope50_w)

    # ATR
    tr = pd.concat([
        df["High"] - df["Low"],
        (df["High"] - df["Close"].shift()).abs(),
        (df["Low"] - df["Close"].shift()).abs()
    ], axis=1).max(axis=1)
    df["ATR"] = tr.rolling(14).mean()

    df["ATR_PCT"] = df["ATR"] / df["Close"]
    df["ATR_PCT_PCTL"] = df["ATR_PCT"].rolling(252).rank(pct=True)
    df["ATR_MULT"] = df["ATR_PCT_PCTL"].apply(atr_mult_from_percentile)

    # Bollinger
    bb_mid = df["Close"].rolling(20).mean()
    bb_std = df["Close"].rolling(20).std()
    df["BB_MID"] = bb_mid
    df["BB_LOWER"] = bb_mid - 2 * bb_std
    df["BB_UPPER_MID"] = bb_mid +  bb_std
    df["BB_LOWER_MID"] = bb_mid -  bb_std

    df["VOL20DMA"] = df["Volume"].rolling(20).mean()
    
    # RSI (14)
    delta = df["Close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()

    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))

    return df

In [4]:
def calculate_cagr(equity_curve):
    total_years = (equity_curve.index[-1] - equity_curve.index[0]).days / 365.25
    if total_years <= 0:
        return 0
    return (equity_curve.iloc[-1] / equity_curve.iloc[0]) ** (1 / total_years) - 1

def calculate_max_drawdown(equity_curve):
    rolling_max = equity_curve.cummax()
    drawdown = (equity_curve - rolling_max) / rolling_max
    return abs(drawdown.min())

def calculate_sharpe(returns, risk_free_rate=0.00):
    excess = returns - risk_free_rate / 252
    if excess.std() == 0:
        return 0
    return (excess.mean() / excess.std()) * np.sqrt(252)

def calculate_sortino(returns, risk_free_rate=0.00):
    downside = returns[returns < 0]
    if downside.std() == 0:
        return 0
    excess = returns.mean() - risk_free_rate / 252
    return (excess / downside.std()) * np.sqrt(252)

def calculate_beta_and_alpha(portfolio_returns: pd.Series, index):
    rf = 0.00  # example: 6% annual RF
    rf_daily = rf / 252
    
    market = yf.download(
        index,
        start=portfolio_returns.index.min(),
        end=portfolio_returns.index.max(),
        progress=False
    )
    market_df = modifications(market)

    # Prefer Adj Close if present
    price_col = "Adj Close" if "Adj Close" in market_df.columns else "Close"
    market_returns = market_df[price_col].pct_change().dropna()
    # Align dates
    df = pd.concat(
        [portfolio_returns, market_returns],
        axis=1,
        join="inner"
    )
    df.columns = ["portfolio", "market"]
    # Excess returns
    excess_port = df["portfolio"] - rf_daily
    excess_mkt = df["market"] - rf_daily

    # Beta
    beta = np.cov(excess_port, excess_mkt)[0, 1] / np.var(excess_mkt)
    #beta = np.polyfit(data['nifty_ret'], data['portfolio_ret'], 1)[0]
    
    # Jensen's alpha (daily)
    alpha_daily = excess_port.mean() - beta * excess_mkt.mean()
    # Annualize alpha
    jensens_alpha = alpha_daily * 252
    
    return beta, jensens_alpha

def calculate_score(CAGR, sortino, max_dd, sharpe):
    # Reject bad strategies
    if CAGR <= 0 or sortino <= 0 or sharpe <=0 or max_dd <= 0:
        return -np.inf

    # Composite objective
    #score = (sortino * CAGR) / max_dd
    score = (sharpe * CAGR) / max_dd
    #score = cagr / max_dd
    return score

In [5]:
def backtest_strategy(
    stock,
    df,
    slope50_window,
    slope200_window1,
    slope200_window2,
    bb_price,
    cc_price,
    rsi,
    initial_capital=25000
):
    df = build_indicators(df, slope50_window, slope200_window1, slope200_window2)
    capital = initial_capital
    in_position = False
    shares = 0
    entry_price = None
    trailing_stop = None
    buy_date = None
    total_abs_dates = 0
    stock_trades = []
    risk_free_rate = .05
    rf_daily = risk_free_rate / 252

    equity_dates = []
    equity_values = []

    # <<< LIVE SIGNAL TRACKER
    today_signal = None
    last_date = df.index[-1]

    for i in range(max(slope50_window, slope200_window1), len(df)):

        TRADE_START_DATE = pd.Timestamp("2024-12-01")
        date = df.index[i]
        if date < TRADE_START_DATE:
            continue
            
        price = df["Close"].iloc[i]
        high_price = df["High"].iloc[i]
        low_price = df["Low"].iloc[i]
        
        if bb_price == "BB_LM0.01":
            ind_price = df["BB_LOWER_MID"].iloc[i]*1.01
        elif bb_price == "BB_MID":
            ind_price = df["BB_MID"].iloc[i]
        elif bb_price == "BB_LOW":
            ind_price = df["BB_LOWER"].iloc[i]*1.02

        if cc_price == "DMA50":
            ind2_price = df["DMA50"].iloc[i]*1.02

        # ---------- UPDATE EQUITY (MARK TO MARKET) ----------
        if in_position:
            equity = shares * price
        else:
            #capital = capital + capital*rf_daily
            equity = capital
        
        if date >= TRADE_START_DATE:
            equity_dates.append(date)
            equity_values.append(equity)

        # ---------- ENTRY ----------
        if not in_position:
            if (
                df["EMA50_SLOPE"].iloc[i] > 0 and
                df["DMA200_SLOPE1"].iloc[i] < 0 and
                price <= ind_price
            ):
                if(low_price<= ind_price <= high_price):
                    exec_price = ind_price
                else:
                    exec_price = price
                shares = capital /exec_price
                entry_price = exec_price
                trailing_stop = exec_price - df["ATR_MULT"].iloc[i] * df["ATR"].iloc[i]
                in_position = True
                buy_date = date

                 # <<< CAPTURE TODAY BUY
                if date == last_date:
                    today_signal = pd.Series({
                        "stock": stock,
                        "signal": "BUY",
                        "price": round(exec_price, 2),
                        "date": date
                    })

        # ---- STRATEGY 2: TREND CONTINUATION ENTRY ----
            if (
               df["DMA200_SLOPE2"].iloc[i] > 0 and       # Strong uptrend
                df["RSI"].iloc[i] > rsi and            # Momentum
                df["Volume"].iloc[i] > df["VOL20DMA"].iloc[i] and
                price <= ind2_price                   # Pullback entry
            ):
                if(low_price<= ind2_price   <= high_price):
                    exec_price = ind2_price 
                else:
                    exec_price = price
                shares = capital /exec_price
                entry_price = exec_price
                trailing_stop = exec_price - df["ATR_MULT"].iloc[i] * df["ATR"].iloc[i]
                in_position = True
                buy_date = date

                 # <<< CAPTURE TODAY BUY
                if date == last_date:
                    today_signal = pd.Series({
                        "stock": stock,
                        "signal": "BUY",
                        "price": round(exec_price, 2),
                        "date": date
                    })

        # ---------- EXIT (ATR TRAILING STOP) ----------
        elif in_position:
            if price < trailing_stop:
                capital = shares * trailing_stop
                shares = 0
                in_position = False
                total_abs_dates += (date - buy_date).days
                stock_trades.append({
                    "stock": stock,
                    "entry_date": pd.to_datetime(buy_date),
                    "exit_date": pd.to_datetime(date)
                })

                # <<< CAPTURE TODAY SELL
                if date == last_date:
                    today_signal = pd.Series({
                        "stock": stock,
                        "signal": "SELL",
                        "price": round(trailing_stop, 2),
                        "date": date,
                        "entry_date": pd.to_datetime(buy_date),
                        "entry_price": entry_price
                    })

                entry_price = None
                trailing_stop = None
                buy_date = None
            else:
                new_stop = price - df["ATR_MULT"].iloc[i] * df["ATR"].iloc[i]
                trailing_stop = max(trailing_stop, new_stop)

    equity_curve = pd.Series(equity_values, index=equity_dates)
    stock_trades_df = pd.DataFrame(stock_trades)

    return equity_curve, total_abs_dates, stock_trades_df, today_signal

In [6]:
def backtest_params_on_stocks(
    stocks,
    slope50_window,
    slope200_window1,
    slope200_window2,
    bb_price,
    cc_price,
    rsi,
    initial_capital=25000
):
    all_returns = []
    all_abs_dates = []
    trades_df = pd.DataFrame(
    columns=["stock", "entry_date", "exit_date"]
    )
    signals = []

    for stock in stocks:
        try:
            df = download_data(stock)
            df = modifications(df)

            equity, abs_dates, stock_trades_df, signal = backtest_strategy(
                stock,
                df,
                slope50_window,
                slope200_window1,
                slope200_window2,
                bb_price,
                cc_price,
                rsi,
                initial_capital=initial_capital
            )

            returns = equity.pct_change().dropna()
            all_returns.append(returns)
            all_abs_dates.append(abs_dates)
            trades_df = pd.concat(
                [trades_df, stock_trades_df],
                ignore_index=True
            )
            if signal is not None:
                signals.append(signal)

        except Exception as e:
            print(f"⚠️ Skipping {stock}: {e}")

    if len(all_returns) == 0:
        return None, None

    # Align returns by date (important)
    combined_returns = pd.concat(all_returns, axis=1).mean(axis=1)

    # Rebuild combined equity curve
    combined_equity = (1 + combined_returns).cumprod() * initial_capital

    avg_abs_days = np.mean(all_abs_dates)

    print(bb_price, cc_price, slope50_window, slope200_window1, slope200_window2, rsi)
    print(f"Average absolute holding days: {avg_abs_days:.2f}")

    # create a daily date range
    all_days = pd.date_range(
        trades_df.entry_date.min(),
        trades_df.exit_date.max(),
        freq="B"
    )

    open_positions = []

    for day in all_days:
        count = (
            (trades_df.entry_date <= day) &
            (trades_df.exit_date > day)
        ).sum()
        open_positions.append(count)

    open_positions = pd.Series(open_positions, index=all_days)

    max_concurrent_positions = open_positions.max()
    avg_concurrent_positions = open_positions.mean()

    print(f"max concurrent_positions: {max_concurrent_positions:.2f}")
    print(f"Average concurrent positions: {avg_concurrent_positions:.2f}")

    signals_df = pd.DataFrame(signals)
    
    return combined_equity, combined_returns, signals_df

In [7]:
stocks = ['RELIANCE.NS', 'HDFCBANK.NS', 'TITAN.NS', 'COALINDIA.NS', 'NTPC.NS', 'IDEA.NS', 'ZEEL.NS', 'ADANIPOWER.NS', 'ACE.NS', 'AFFLE.NS', 'NESCO.NS',
          'ANGELONE.NS','KINGFA.NS', 'MTARTECH.NS', 'TATAPOWER.NS', 'TATASTEEL.NS',  'TECHM.NS', 'TIMKEN.NS', 'NAZARA.NS', 'POLYCAB.NS', 'PRITI.NS', 
          'SAIL.NS', 'SBICARD.NS', 'BLS.NS', 'LTIM.NS', 'DELTACORP.NS',  'EASEMYTRIP.NS', 'GRSE.NS', 'HAPPSTMNDS.NS', 'INDIGOPNTS.NS', 
          'IRCTC.NS', 'KOTAKBANK.NS', 'LIKHITHA.NS', 'MAPMYINDIA.NS', 'TCS.NS', 'ITC.NS', 'YESBANK.NS', 'ADANIPORTS.NS', 'BAJAJ-AUTO.NS', 'CRISIL.NS']

nifty50stocks = [
"ADANIENT.NS", "ADANIPORTS.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS", "AXISBANK.NS","BAJAJ-AUTO.NS", "BAJFINANCE.NS", "BAJAJFINSV.NS", "BEL.NS", "BHARTIARTL.NS",
"CIPLA.NS", "COALINDIA.NS", "DRREDDY.NS", "EICHERMOT.NS", "ETERNAL.NS","GRASIM.NS", "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HINDALCO.NS",
"HINDUNILVR.NS", "ICICIBANK.NS", "INFY.NS", "INDIGO.NS", "ITC.NS","JIOFIN.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS", "M&M.NS",
"MARUTI.NS", "MAXHEALTH.NS", "NESTLEIND.NS", "NTPC.NS", "ONGC.NS","POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SHRIRAMFIN.NS", "SBIN.NS",
"SUNPHARMA.NS", "TCS.NS", "TATACONSUM.NS", "TATASTEEL.NS","TECHM.NS", "TITAN.NS", "TRENT.NS", "ULTRACEMCO.NS", "WIPRO.NS"
]

midcap100stocks = ["ACC.NS", "ATGL.NS", "ABCAPITAL.NS", "ABFRL.NS", "ALKEM.NS", "APLAPOLLO.NS", "APOLLOTYRE.NS", "ASHOKLEY.NS", "ASTRAL.NS",
"AUBANK.NS", "AUROPHARMA.NS", "BANDHANBNK.NS", "BANKINDIA.NS", "MAHABANK.NS", "BDL.NS", "BHARATFORG.NS", "BHEL.NS", "BHARTIHEXA.NS", "BIOCON.NS", 
"BRITANNIA.NS", "COCHINSHIP.NS", "COFORGE.NS", "COLPAL.NS", "CONCOR.NS", "CUMMINSIND.NS", "DIXON.NS", "ESCORTS.NS", "EXIDEIND.NS", "FEDERALBNK.NS", 
"NYKAA.NS", "GLENMARK.NS", "GODREJPROP.NS", "HDFCAMC.NS", "HINDPETRO.NS", "HINDZINC.NS", "HUDCO.NS", "IDFCFIRSTB.NS", "INDIANB.NS", 
"IEX.NS", "IGL.NS", "INDUSTOWER.NS", "IRB.NS", "IRCTC.NS", "JUBLFOOD.NS", "KALYANKJIL.NS", "KPITTECH.NS", "LTF.NS", "LICHSGFIN.NS", "LUPIN.NS", 
"M&MFIN.NS", "MANKIND.NS", "MARICO.NS", "MFSL.NS", "MAXHEALTH.NS", "MAZDOCK.NS", "MOTILALOFS.NS", "MPHASIS.NS", "MRF.NS", "MUTHOOTFIN.NS",
"NATIONALUM.NS", "NHPC.NS", "NMDC.NS", "NTPCGREEN.NS", "OBEROIRLTY.NS", "OIL.NS", "OLAELEC.NS", "PAYTM.NS", "OFSS.NS", "PAGEIND.NS", "PATANJALI.NS",
"PERSISTENT.NS", "PETRONET.NS", "PHOENIXLTD.NS", "PIIND.NS", "POLYCAB.NS", "PREMIERENE.NS", "PRESTIGE.NS", "RVNL.NS", "SBICARD.NS",
"SJVN.NS", "SOLARINDS.NS", "SONACOMS.NS", "SRF.NS", "SAIL.NS", "SUPREMEIND.NS", "SUZLON.NS", "TATACOMM.NS", "TATAELXSI.NS", "TATATECH.NS", "TORNTPOWER.NS", 
"TIINDIA.NS", "UNIONBANK.NS", "UPL.NS", "VMM.NS", "IDEA.NS", "VOLTAS.NS", "WAAREEENER.NS", "YESBANK.NS"]

univstocks = [
"TCS.NS", "INFY.NS", "MARUTI.NS", "HCLTECH.NS", "ITC.NS", "SUNPHARMA.NS","BEL.NS", "ASIANPAINT.NS", "BAJAJ-AUTO.NS", "NESTLEIND.NS", "EICHERMOT.NS", 
"TRENT.NS",  "CIPLA.NS",
    
"BOSCHLTD.NS", "CGPOWER.NS", "DIVISLAB.NS", "HAVELLS.NS", "LTIM.NS", "PIDILITIND.NS", "TORNTPHARM.NS", "UNITDSPR.NS", "ZYDUSLIFE.NS",
    
"BRITANNIA.NS", "POLYCAB.NS", "SOLARINDS.NS", "CUMMINSIND.NS", "HDFCAMC.NS", "MARICO.NS", "MAZDOCK.NS", "PERSISTENT.NS", 
"NMDC.NS", "DIXON.NS", "OFSS.NS", "NATIONALUM.NS", "HINDPETRO.NS", "APLAPOLLO.NS", "MPHASIS.NS", "IRCTC.NS", "LTF.NS", "KPITTECH.NS",
"ALKEM.NS", "COFORGE.NS", "LUPIN.NS", "TIINDIA.NS",
    
"GODFRYPHLP.NS", "NBCC.NS", "GRSE.NS", "HBLENGINE.NS", "LALPATHLAB.NS", "CASTROLIND.NS", "CAMS.NS", "TRITURBINE.NS", "BLS.NS",
"NEWGEN.NS", "ACE.NS","BSOFT.NS", "HSCL.NS", "HINDCOPPER.NS", "MCX.NS", "RAILTEL.NS", "RITES.NS", "WELCORP.NS", "ZENSARTECH.NS"]

newunivstocks = [
 "TCS.NS", "INFY.NS", "HCLTECH.NS", "SUNPHARMA.NS", "NESTLEIND.NS", "EICHERMOT.NS", "TRENT.NS",  "CIPLA.NS", "HINDUNILVR.NS",
    
"BOSCHLTD.NS", "CGPOWER.NS", "DIVISLAB.NS",  "LTIM.NS", "PIDILITIND.NS", "TORNTPHARM.NS", "ZYDUSLIFE.NS", "VBL.NS",
    
"BRITANNIA.NS", "POLYCAB.NS", "SOLARINDS.NS", "HDFCAMC.NS", "MARICO.NS", "PERSISTENT.NS", "DIXON.NS", "OFSS.NS", "MPHASIS.NS","KPITTECH.NS",
"COFORGE.NS", "LUPIN.NS", "TIINDIA.NS", "INDUSTOWER.NS", "DRREDDY.NS", "WAAREEENER.NS", "GVT&D.NS", "COROMANDEL.NS", "SCHAEFFLER.NS",
"ABBOTINDIA.NS", "NAM-INDIA.NS",
    
"GRSE.NS", "HBLENGINE.NS", "LALPATHLAB.NS", "CASTROLIND.NS", "BLS.NS", "HINDCOPPER.NS", "MCX.NS", "RAILTEL.NS", "HEXT.NS",
"KEI.NS", "3MINDIA.NS", "TATAELXSI.NS", "PREMIERENE.NS", "MSUMI.NS", "FORCEMOT.NS", "GILLETTE.NS", "ANANDRATHI.NS", "ABSLAMC.NS",
"ECLERX.NS", "ASTRAZEN.NS", "DOMS.NS", "CAPLIPOINT.NS", "IGIL.NS", "GABRIEL.NS", "SHRIPISTON.NS", "FINEORG.NS", "INDIAMART.NS", "ENGINERSIN.NS",
"TDPOWERSYS.NS", "INOXINDIA.NS", "PRUDENT.NS", "WAAREERTL.NS", "SANOFICONR.NS", "JYOTHYLAB.NS", "TANLA.NS", "KINGFA.NS", "QPOWER.NS",
"CIGNITITEC.NS", "SHILCTECH.NS", "TIMEX.BO"]

smallcap100stocks = [
"AADHARHFC.NS", "AARTIIND.NS", "ACE.NS", "ABREL.NS", "AEGISLOG.NS", "AFCONS.NS","AFFLE.NS", "AMBER.NS", "ANANTRAJ.NS", "ANGELONE.NS", "ASTERDM.NS",
"ATUL.NS", "BATAINDIA.NS", "BEML.NS", "BSOFT.NS", "BLS.NS", "BRIGADE.NS","CASTROLIND.NS", "CESC.NS", "CHAMBLFERT.NS", "CAMS.NS", "CREDITACC.NS",
"CGPOWER.NS", "CYIENT.NS", "DATAPATTNS.NS", "DELHIVERY.NS", "DEVYANI.NS","LALPATHLAB.NS", "FSL.NS", "FIVESTAR.NS", "GRSE.NS", "GODIGIT.NS",
"GODFRYPHLP.NS", "GESHIP.NS", "GSPL.NS", "HBLENGINE.NS", "HFCL.NS", "HINDCOPPER.NS", "IDBI.NS", "IFCI.NS", "IIFL.NS",
"INDIAMART.NS", "IEX.NS", "INOXWIND.NS", "IGIL.NS","IRCON.NS", "ITI.NS", "JBMA.NS", "KPITTECH.NS", "KARURVYSYA.NS",
"KAYNES.NS", "KEC.NS", "KFINTECH.NS", "LAURUSLABS.NS", "MGL.NS","MANAPPURAM.NS", "MCX.NS", "NH.NS", "NATCOPHARM.NS", "NAVINFLUOR.NS",
"NBCC.NS", "NCC.NS", "NEULANDLAB.NS", "NEWGEN.NS", "NUVAMA.NS","PCBL.NS", "PGEL.NS", "PNBHOUSING.NS", "POONAWALLA.NS",
"PVRINOX.NS", "RADICO.NS", "RAILTEL.NS", "RKFORGE.NS", "REDINGTON.NS","RPOWER.NS", "RITES.NS", "SAGILITY.NS", "SHYAMMETL.NS",
"SIGNATURE.NS", "SONATSOFTW.NS", "TATACHEM.NS","TTML.NS", "TEJASNET.NS", "RAMCOCEM.NS", "TITAGARH.NS", "TRIDENT.NS",
"TRITURBINE.NS", "WELCORP.NS", "WELSPUNLIV.NS", "ZENTEC.NS", "ZENSARTECH.NS"
]

In [18]:
#live trading signal
#github with all the poc and documentation

stocks_shuffled = univstocks.copy()
random.shuffle(stocks_shuffled)

train_size = int(1*len(univstocks))   # 80% train, 20% test

train_stocks = stocks_shuffled[:train_size]
test_stocks  = stocks_shuffled[train_size:]

param_grid = {
    "slope50_window": [7],
    "slope200_window1": [10],
    "slope200_window2": [13],
    "bb_price": ["BB_MID"],
    "cc_price": ["DMA50"],
    "rsi": [60]
}

results = []

for params in itertools.product(*param_grid.values()):
    p = dict(zip(param_grid.keys(), params))

    equity, returns, signals_df = backtest_params_on_stocks(
        train_stocks,
        p["slope50_window"],
        p["slope200_window1"],
        p["slope200_window2"],
        p["bb_price"],
        p["cc_price"],
        p["rsi"]
    )

    if equity is None:
        continue

    cagr = calculate_cagr(equity)
    max_dd = calculate_max_drawdown(equity)
    sharpe = calculate_sharpe(returns)
    sortino = calculate_sortino(returns)
    beta, jensens_alpha = calculate_beta_and_alpha (returns, "^CRSLDX") #"^NSEI", "NIFTY_MIDCAP_100.NS", "^CNXSC", "^CRSLDX"

    score = calculate_score(cagr, sortino, max_dd, sharpe)

    results.append({
        **p,
        "CAGR": cagr,
        "MaxDD": max_dd,
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Score": score,
        "Beta": beta,
        "Jensens_alpha": jensens_alpha
    })

results_df = pd.DataFrame(results)

C:\Users\saksh\AppData\Local\Temp\ipykernel_21752\719025167.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start="2024-03-01", progress=False)
C:\Users\saksh\AppData\Local\Temp\ipykernel_21752\748054319.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trades_df = pd.concat(
C:\Users\saksh\AppData\Local\Temp\ipykernel_21752\719025167.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start="2024-03-01", progress=False)
C:\Users\saksh\AppData\Local\Temp\ipykernel_21752\719025167.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start="2024-03-01", progress=Fal

BB_MID DMA50 7 10 13 60
Average absolute holding days: 111.34
max concurrent_positions: 62.00
Average concurrent positions: 27.07


C:\Users\saksh\AppData\Local\Temp\ipykernel_21752\2958513805.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market = yf.download(


In [19]:
results_df = results_df.sort_values("Jensens_alpha", ascending=False)
best_params = results_df.iloc[0]
print(best_params)
print(signals_df)

slope50_window             7
slope200_window1          10
slope200_window2          13
bb_price              BB_MID
cc_price               DMA50
rsi                       60
CAGR                0.028892
MaxDD               0.048423
Sharpe              0.530937
Sortino             0.677175
Score               0.316787
Beta                0.310645
Jensens_alpha       0.032285
Name: 0, dtype: object
     stock signal  price       date entry_date  entry_price
0  IIFL.NS   SELL  618.5 2026-01-22 2025-10-03   463.630802


In [96]:
equity, returns = backtest_params_on_stocks(
        test_stocks,
        best_params["slope50_window"],
        best_params["slope200_window1"],
        best_params["slope200_window2"],
        best_params["bb_price"],
        best_params["cc_price"],
        best_params["rsi"]
    )

cagr = calculate_cagr(equity)
max_dd = calculate_max_drawdown(equity)
sharpe = calculate_sharpe(returns)
sortino = calculate_sortino(returns)

score = calculate_score(cagr, sortino, max_dd, sharpe)

result = []
result.append({
    "CAGR": cagr,
    "MaxDD": max_dd,
    "Sharpe": sharpe,
    "Sortino": sortino,
    "Score": score
})

result_df = pd.DataFrame(result)

C:\Users\saksh\AppData\Local\Temp\ipykernel_18480\11056871.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start='2022-03-01', end='2025-12-31')
[*********************100%***********************]  1 of 1 completed


SBILIFE.NS


C:\Users\saksh\AppData\Local\Temp\ipykernel_18480\2326466076.py:114: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(equity_curve[-1])
C:\Users\saksh\AppData\Local\Temp\ipykernel_18480\1075970652.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trades_df = pd.concat(
C:\Users\saksh\AppData\Local\Temp\ipykernel_18480\11056871.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock, start='2022-03-01', end='2025-12-31')


43412.56480585541


[*********************100%***********************]  1 of 1 completed


...

34654.11066809261


[*********************100%***********************]  1 of 1 completed


LT.NS


C:\Users\saksh\AppData\Local\Temp\ipykernel_18480\2326466076.py:114: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(equity_curve[-1])


32152.42012249189
BB_MID DMA50 7 10 10 60
Average absolute holding days: 370.60
max concurrent_positions: 8.00
Average concurrent positions: 3.50


In [3]:
BOT_TOKEN = "7777418422:AAFp4qOqAZcQxEZx7IS0nEpuQaOEyrnKMdI"
CHAT_ID = "8335782875"

def send_telegram(message):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {
        "chat_id": CHAT_ID,
        "text": message
    }
    r = requests.post(url, data=payload)
    print(r.json())

send_telegram("signals_df")


{'ok': True, 'result': {'message_id': 4, 'from': {'id': 7777418422, 'is_bot': True, 'first_name': 'Setter', 'username': 'set_it_signal_bot'}, 'chat': {'id': 8335782875, 'first_name': 'Saksham', 'type': 'private'}, 'date': 1768326649, 'text': '✅ Telegram bot connected successfully!'}}
